In [59]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
from rasa.train import train_nlu, train_async
prefix='/pi/ws/sagas-ai/nlu_multilang'
train_nlu(
    config=f"{prefix}/config_en.yml",
    nlu_data=f"{prefix}/en/",
    output=f'{prefix}/models',
    fixed_model_name='en_current',
    persist_nlu_training_data=True
)

Training NLU model...


/Users/xiaofeiwu/miniconda3/envs/rasa/lib/python3.6/site-packages/rasa/nlu/training_data/training_data.py:341: UserWarning: Entity 'breakfast-included' has only 1 training examples! minimum is 2, training may fail.
  f"Entity '{entity_type}' has only {count} training examples! "
Epochs: 100%|██████████| 300/300 [00:22<00:00, 14.43it/s, loss=0.812, acc=0.996]


NLU model training completed.
Your Rasa model is trained and saved at '/pi/ws/sagas-ai/nlu_multilang/models/en_current.tar.gz'.


'/pi/ws/sagas-ai/nlu_multilang/models/en_current.tar.gz'

In [43]:
from rasa.core.agent import Agent, load_agent
# from rasa.nlu.model import Interpreter
path='/pi/ws/sagas-ai/nlu_multilang/models/en_current.tar.gz'
# interpreter = Interpreter.load(path)
agent = Agent.load(path)

INFO:tensorflow:Restoring parameters from /var/folders/fv/7k1qk5v11dn33sdcngv2wbnm0000gn/T/tmp5weag0vd/nlu/component_6_EmbeddingIntentClassifier.ckpt



In [54]:
interpreter=agent.interpreter

In [57]:
text='hi'
result = await interpreter.parse(text)
result

{'intent': {'name': 'greet', 'confidence': 0.9759674668312073},
 'entities': [],
 'intent_ranking': [{'name': 'greet', 'confidence': 0.9759674668312073},
  {'name': 'deny', 'confidence': 0.003993101883679628},
  {'name': 'joke', 'confidence': 0.0035071619786322117},
  {'name': 'name', 'confidence': 0.003050952684134245},
  {'name': 'bot_challenge', 'confidence': 0.0023912435863167048},
  {'name': 'thank', 'confidence': 0.0023714040871709585},
  {'name': 'request_restaurant', 'confidence': 0.0017789762932807207},
  {'name': 'query_knowledge_base', 'confidence': 0.0017731953412294388},
  {'name': 'affirm', 'confidence': 0.0013133026659488678},
  {'name': 'stop', 'confidence': 0.0009981970069929957}],
 'text': 'hi'}

In [45]:
# await agent.handle_text("hello")
from rasa.core.trackers import DialogueStateTracker
from rasa.core.slots import Slot
from pprint import pprint

tracker = DialogueStateTracker.from_dict("1", [], [Slot("requested_language")])
# we'll expect this value 'en' to be part of the result from the interpreter
tracker._set_slot("requested_language", "en")
result = await agent.parse_message_using_nlu_interpreter("lunch?", tracker)
pprint(result)

{'entities': [],
 'intent': {'confidence': 0.15044686198234558, 'name': 'name'},
 'intent_ranking': [{'confidence': 0.15044686198234558, 'name': 'name'},
                    {'confidence': 0.1345827579498291, 'name': 'inform'},
                    {'confidence': 0.0954357162117958, 'name': 'thank'},
                    {'confidence': 0.06806426495313644, 'name': 'affirm'},
                    {'confidence': 0.06687286496162415,
                     'name': 'bot_challenge'},
                    {'confidence': 0.06323861330747604, 'name': 'goodbye'},
                    {'confidence': 0.061934877187013626, 'name': 'greet'},
                    {'confidence': 0.05717906728386879, 'name': 'chitchat'},
                    {'confidence': 0.0568377748131752, 'name': 'stop'},
                    {'confidence': 0.05530387535691261, 'name': 'deny'}],
 'text': 'lunch?'}


/Users/xiaofeiwu/miniconda3/envs/rasa/lib/python3.6/site-packages/rasa/core/processor.py:311: UserWarning: Interpreter parsed an intent 'name' that is not defined in the domain.
  f"Interpreter parsed an intent '{intent}' "


In [3]:
from rasa.core.trackers import DialogueStateTracker
from rasa.core.interpreter import RasaNLUHttpInterpreter
from rasa.utils.endpoints import EndpointConfig
from rasa.core.slots import Slot
from pprint import pprint

DEFAULT_SERVER_PORT = 5008
DEFAULT_SERVER_FORMAT = "{}://localhost:{}"
DEFAULT_SERVER_URL = DEFAULT_SERVER_FORMAT.format("http", DEFAULT_SERVER_PORT)

async def nlu_parse(url, message):
    tracker = DialogueStateTracker.from_dict("1", [], [Slot("requested_language")])
    # we'll expect this value 'en' to be part of the result from the interpreter
    tracker._set_slot("requested_language", "en")
    inte=RasaNLUHttpInterpreter(EndpointConfig(url))
    result = await inte.parse(message, tracker=tracker)
    return result

result=await nlu_parse(DEFAULT_SERVER_URL, 'hi')
pprint(result)

{'entities': [],
 'intent': {'confidence': 0.9759674668, 'name': 'greet'},
 'intent_ranking': [{'confidence': 0.9759674668, 'name': 'greet'},
                    {'confidence': 0.0039931019, 'name': 'deny'},
                    {'confidence': 0.003507162, 'name': 'joke'},
                    {'confidence': 0.0030509527, 'name': 'name'},
                    {'confidence': 0.0023912436, 'name': 'bot_challenge'},
                    {'confidence': 0.0023714041, 'name': 'thank'},
                    {'confidence': 0.0017789763, 'name': 'request_restaurant'},
                    {'confidence': 0.0017731953,
                     'name': 'query_knowledge_base'},
                    {'confidence': 0.0013133027, 'name': 'affirm'},
                    {'confidence': 0.000998197, 'name': 'stop'}],
 'text': 'hi'}


In [11]:
result=await nlu_parse('http://localhost:5009/simulate/en', 'hi')
pprint(result)

{'entities': [], 'intent': {'confidence': 0.0, 'name': ''}, 'text': 'hi'}


In [12]:
result=await nlu_parse('http://localhost:5009/nlu/en', 'hi')
pprint(result)

{'entities': [],
 'intent': {'confidence': 0.9759674668, 'name': 'greet'},
 'intent_ranking': [{'confidence': 0.9759674668, 'name': 'greet'},
                    {'confidence': 0.0039931019, 'name': 'deny'},
                    {'confidence': 0.003507162, 'name': 'joke'},
                    {'confidence': 0.0030509527, 'name': 'name'},
                    {'confidence': 0.0023912436, 'name': 'bot_challenge'},
                    {'confidence': 0.0023714041, 'name': 'thank'},
                    {'confidence': 0.0017789763, 'name': 'request_restaurant'},
                    {'confidence': 0.0017731953,
                     'name': 'query_knowledge_base'},
                    {'confidence': 0.0013133027, 'name': 'affirm'},
                    {'confidence': 0.000998197, 'name': 'stop'}],
 'text': 'hi'}


In [14]:
?os.listdir

In [17]:
import io_utils
io_utils.list_files(f"{prefix}/en/")

['/pi/ws/sagas-ai/nlu_multilang/en/knowledgebase.md',
 '/pi/ws/sagas-ai/nlu_multilang/en/common.md',
 '/pi/ws/sagas-ai/nlu_multilang/en/nlu_data.md']

In [60]:
# import os
import time
import json_utils

mods=('de', 'en', 'es', 'fr', 'ru', 'corpus')

def get_timestamps():
    last_timestamps={}
    for mod in mods:
        folder = f"{prefix}/{mod}/"
        # print(folder)
        # print(os.listdir(folder))
        files = [(f, os.path.getmtime(f)) for f in io_utils.list_files(folder) 
                        if os.path.isfile(f)]
        # print(len(files))
        files = sorted(files, key=lambda x: x[1], reverse=True)
        last_modified_file = None if len(files) == 0 else files[0][0]
        if last_modified_file is not None:
            mark=os.path.getmtime(last_modified_file)
            # print(last_modified_file, mark)
            last_timestamps[mod]=mark
    return last_timestamps

last_timestamps=get_timestamps()
print(last_timestamps)
json_utils.write_json_to_file('./nlu_multilang/last_timestamps.json', last_timestamps)

{'de': 1566114480.0, 'en': 1576014032.2032847, 'es': 1566285239.0, 'fr': 1576183805.222966, 'ru': 1566288205.0, 'corpus': 1576184112.656068}


In [40]:
before=json_utils.read_json_file('./nlu_multilang/last_timestamps.json')
current=get_timestamps()
modified=[]
for item in current:
    if current[item]-before[item]>1:
        modified.append(item)
print(modified)

['en', 'corpus']


In [24]:
t=os.path.getmtime(last_modified_file)
t-time.time()

-34.68090867996216

In [4]:
await train_async(
        domain={},
        config=f"{prefix}/config_en.yml",
        training_files=f"{prefix}/en/",
        output_path=f'{prefix}/models',
        fixed_model_name='en_current',
    )

Training NLU model...


/Users/xiaofeiwu/miniconda3/envs/rasa/lib/python3.6/site-packages/rasa/nlu/training_data/training_data.py:341: UserWarning: Entity 'breakfast-included' has only 1 training examples! minimum is 2, training may fail.
  f"Entity '{entity_type}' has only {count} training examples! "
Epochs: 100%|██████████| 300/300 [00:21<00:00, 14.79it/s, loss=0.670, acc=0.998]


NLU model training completed.


/Users/xiaofeiwu/miniconda3/envs/rasa/lib/python3.6/site-packages/rasa/importers/rasa.py:70: UserWarning: Loading domain from '{}' failed. Using empty domain. Error: 'No domain file was specified. Please specify a path to a valid domain file.'
  f"Loading domain from '{self._domain_path}' failed. Using "


Your Rasa model is trained and saved at '/pi/ws/sagas-ai/nlu_multilang/models/en_current.tar.gz'.
Core training was skipped because no valid domain file was found. Only an nlu-model was created.Please specify a valid domain using '--domain' argument or check if the provided domain file exists.


'/pi/ws/sagas-ai/nlu_multilang/models/en_current.tar.gz'